## Version Information

<table align="left">
  <thead>
    <tr>
      <th>Component</th>
      <th>Version</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>python</td>
      <td>3.11.5</td>
    </tr>
    <tr>
      <td>pytket</td>
      <td>1.27.0</td>
    </tr>
  </tbody>
</table>

</table>


In [1]:
from pytket import Circuit

c = Circuit(2, 2) # define a circuit with 2 qubits and 2 bits
c.H(0)            # add a Hadamard gate to qubit 0
c.Rz(0.25, 0)     # add an Rz gate of angle 0.25*pi to qubit 0
c.CX(1,0)         # add a CX gate with control qubit 1 and target qubit 0
c.measure_all()   # measure qubits 0 and 1, recording the results in bits 0 and 1

[H q[0]; Rz(0.25) q[0]; CX q[1], q[0]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [2]:
import math
import numpy as np
from pytket import Circuit
from pytket.extensions.qiskit import AerBackend, tk_to_qiskit, qiskit_to_tk
from pytket.passes import FullPeepholeOptimise
from pytket.circuit.display import render_circuit_jupyter
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

def find_all_as(N):
    ns = []
    for n in range(2, N):
        if math.gcd(n, N) == 1:
            ns.append(n)
    return ns

N = 7
a = 2
n_len = math.ceil(math.log2(N))
m_len = 2 * n_len

# Create a pytket circuit
circuit = Circuit(m_len + n_len)

# Apply Hadamard gates to the first m_len qubits
for q in range(m_len):
    circuit.H(q)

# Apply X gate to the m_len qubit
circuit.X(m_len)

# Find all 'a's that are coprime to N
ns = find_all_as(N)

if a not in ns:
    raise ValueError(f"'a' must be one of {ns}")

# Apply controlled-U gates
for q in range(m_len):
    power = 2**q
    result = pow(a, power, N)
    for i, bit in enumerate(format(result ^ 1, f'0{n_len}b')):
        if bit == '1':
            circuit.CX(q, i + m_len)

# Apply inverse QFT
def qft_inv(circ, qubits):
    n = len(qubits)
    for j in range(n):
        for k in range(j):
            circ.CRz(-math.pi / (2 ** (j - k)), qubits[j], qubits[k])
        circ.H(qubits[j])
    for i in range(n//2):
        circ.SWAP(qubits[i], qubits[n-i-1])
    return circ

circuit = qft_inv(circuit, range(m_len))

# Measure the first m_len qubits
circuit.measure_all()

# Display the circuit
render_circuit_jupyter(circuit)

# Optimize the circuit
FullPeepholeOptimise().apply(circuit)

# Convert to Qiskit circuit for simulation
qc = tk_to_qiskit(circuit)

# Simulate the circuit using Aer backend
backend = AerBackend()
compiled_circuit = backend.get_compiled_circuit(circuit)
result = backend.run_circuit(compiled_circuit, n_shots=1000)
counts = result.get_counts()

# Convert counts to a format suitable for plotting
counts_formatted = {format(key, '0' + str(m_len) + 'b'): val for key, val in counts.items()}

# Plot the histogram
plot_histogram(counts_formatted, figsize=(17, 5))
plt.show()


ImportError: cannot import name 'ValidationInfo' from 'pydantic' (C:\Users\ahmed\anaconda3\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)